In [1]:
!pip install pycox

  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2462 sha256=5c1d75b259f886e249d14ea4cf8bc6ffa227b89da1d89324696a7ddb72682e15
  Stored in directory: c:\users\hnguye78\appdata\local\pip\cache\wheels\7b\83\4b\44aa8f2292c0ae17a550c5105454e974fc45a5b2a522d66ae1
Successfully built feather-format


In [1]:
import numpy as np
n_features = 'all'
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10'
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
endpt = 16;
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [ ]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,5258.5,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,5560.5,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,5381.5,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100056526386,0,5511.5,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
4,100061300991,0,5403.5,0,1,200.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4196,416771521620,0,5586.5,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4197,416783315386,0,5563.5,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4198,416796224310,0,5452.5,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,444,445,446,447,448,449,450,451,452,453
0,100016012504,0,5258.5,-1.120553,-0.959853,0.239613,-1.105713,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.774994
1,100023004268,0,5560.5,-1.120553,-0.959853,-0.356702,-0.746751,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.981136
2,100033323702,0,5381.5,-1.120553,-0.959853,1.670768,-0.100620,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,1.668274
3,100056526386,0,5511.5,-1.120553,1.041826,0.627217,-0.674959,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.461855
4,100061300991,0,5403.5,-1.120553,1.041826,0.657033,-0.962128,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.736711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,0.892416,-0.959853,1.402426,0.258341,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,0.774994
4196,416771521620,0,5586.5,-1.120553,1.041826,0.776296,1.622396,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.599283
4197,416783315386,0,5563.5,0.892416,-0.959853,-0.565412,0.330134,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,0.853663,-0.089138,-0.121811,2.767696
4198,416796224310,0,5452.5,0.892416,1.041826,-0.416334,0.114757,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,1.462133


In [9]:
# Outerloop:
for fold in range(7,25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 7:

Using nesterov accelerated gradient
training model
cost: 4.499070, iteration: 0
cost: 4.471881, iteration: 23
cost: 4.434885, iteration: 46
cost: 4.397338, iteration: 69
cost: 4.360056, iteration: 92
cost: 4.323109, iteration: 115
cost: 4.286501, iteration: 138
cost: 4.250228, iteration: 161
cost: 4.214289, iteration: 184
cost: 4.178679, iteration: 207
cost: 4.143396, iteration: 230
cost: 4.108437, iteration: 253
cost: 4.073799, iteration: 276
cost: 4.039479, iteration: 299
cost: 4.005474, iteration: 322
cost: 3.971780, iteration: 345
cost: 3.938396, iteration: 368
cost: 3.905319, iteration: 391
cost: 3.872545, iteration: 414
cost: 3.840072, iteration: 437
cost: 3.807897, iteration: 460
cost: 3.776017, iteration: 483
cost: 3.744429, iteration: 506
cost: 3.713132, iteration: 529
cost: 3.682122, iteration: 552
cost: 3.651396, iteration: 575
cost: 3.620953, iteration: 598
cost: 3.590789, iteration: 621
cost: 3.560902, iteration: 644
cost: 3.531289, iteration: 667
cost: 3.501948,